# Overview of Ontario's Covid-19 Spread
> Insights into the Covid-19 spread in Ontario, Canada.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/test.png

In [1]:
#hide

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import altair as alt

from IPython.display import HTML, display
import pytz

import ipywidgets as widgets

from datetime import datetime, timedelta, timezone

print('Libraries successfully imported.')

Libraries successfully imported.


In [3]:
#hide
#%%time

url = 'https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/455fd63b-603d-4608-8216-7d8647f43350/download/conposcovidloc.csv'
url_2 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/ed270bb8-340b-41f9-a7c6-e8ef587e6d11/download/covidtesting.csv'
url_3 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/8a88fe6d-d8fb-41a3-9d04-f0550a44999f/download/daily_change_in_cases_by_phu.csv'

#geo = gpd.read_file('https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/4f39b02b-47fe-4e66-95b6-e6da879c6910/download/conposcovidloc.geojson') 

src_conpos = pd.read_csv(url, index_col=0, parse_dates=['Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date']).reset_index()
src_testing = pd.read_csv(url_2, index_col=0, parse_dates=['Reported Date']).reset_index()
src_daily = pd.read_csv(url_3, index_col=0, parse_dates=['Date']).reset_index()

# duplicate entry for December 5th, 2020 causing error
src_testing.drop_duplicates(inplace=True)

print('Source files successfully loaded.')

Source files successfully loaded.


In [4]:
#hide
last_date = src_testing['Reported Date'].max()
init_date = src_conpos['Accurate_Episode_Date'].min()
display(HTML("Ontario data set last updated on: " + last_date.strftime("%x")))

In [5]:
#hide_input
src_testing[src_testing['Confirmed Negative'] > 0].tail() # no confirmed negatives after March 29th 2020

testing_latest = src_testing[src_testing['Reported Date'] == src_testing['Reported Date'].max()]

# get the previous day results
testing_delta1 = src_testing[src_testing['Reported Date'] == src_testing['Reported Date'].max() - timedelta(days=1)]

# get the daily percentage +/-
dailypct = src_testing.join(src_testing[['Total Cases', 'Resolved', 'Confirmed Positive', 'Deaths']].pct_change().add_suffix('_pct'))

dailypct = dailypct[dailypct['Reported Date'] == dailypct['Reported Date'].max()]

# apply a percentage format; conventional formatting method throws an error
def percentage_delta(raw):
  # pctfmt = raw.map('{:+.2%}'.format)
  # split = str(pctfmt).split()
  formatted_pct = str(raw.map('{:+.2%}'.format)).split()[1]
  return formatted_pct

# original background colour code: #504e4e, secondary: #585858

display(HTML(
    
    "<div id='number-plate' style = 'background-color: #ececec; padding: 30px; text-align: center; marginTop: 1%; marginBottom: 1%;'>" +

    "<p style='color: black; font-size: 32px'> Ontario Covid-19 Cases to Date</p>"
    
    "<div id='confirmed' style = 'display: inline-block'> <span style='color: black; font-size: 23px;'> Confirmed: " +
    str('{:,.0f}'.format(int(testing_latest['Total Cases']))) + "</span>" + 
    "<br> <span style='color: black; font-size:15px; '> " + 
    str('{0:+,d}'.format(int(testing_latest['Total Cases']) - (int(testing_delta1['Total Cases'])))) +
    ' (' + percentage_delta(dailypct['Total Cases_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<div id='resolved' style = 'display: inline-block'> <span style='color: #228b22; font-size: 23px; margin-left: 20px;'> Resolved: " + 
    str('{:,.0f}'.format(int(testing_latest['Resolved']))) + "</span>" + 
    "<br> <span style='color: black; font-size:15px; '>" + 
    str('{0:+,d}'.format(int(testing_latest['Resolved']) - (int(testing_delta1['Resolved'])))) +
    ' (' + percentage_delta(dailypct['Resolved_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<div id='active' style = 'display: inline-block'> <span style='color: orange; font-size: 23px; margin-left: 20px;'> Active: " + 
    str('{:,.0f}'.format(int(testing_latest['Confirmed Positive']))) +"</span>" + 
    "<br> <span style='color: black; font-size:15px; '>" + 
    str('{0:+,d}'.format(int(testing_latest['Confirmed Positive']) - (int(testing_delta1['Confirmed Positive'])))) +
    ' (' + percentage_delta(dailypct['Confirmed Positive_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<div id='deaths' style = 'display: inline-block'> <span style='color: red; font-size: 23px; margin-left: 20px;'> Deaths: " + 
    str('{:,.0f}'.format(int(testing_latest['Deaths']))) + "</span>" + 
    "<br> <span style='color: black; font-size:15px; '>" + 
    str('{0:+,d}'.format(int(testing_latest['Deaths']) - (int(testing_delta1['Deaths'])))) +
    ' (' + percentage_delta(dailypct['Deaths_pct']) + ')' +
    "</span>" + 
    "</div>" +

    "<br><div id='rates' style = 'display: block; marginTop: 1%;'> " +

    "<div id='deathrate' style = 'display: inline-block'> <span style='color: black;'> Current Death Rate: " +
    "</span>" +
    "<span style = 'color: red'>" +
    str((testing_latest['Deaths'] / testing_latest['Total Cases']).map('{:.2%}'.format)).split()[1] + 
    "</span>" +
    "</div>" +

    "<div id='survivalrate' style = 'display:inline-block'> <span style='color: black; margin-left: 20px;'> Current Survival Rate: " + 
    "</span>" +
    "<span style = 'color: #228b22'>" +
    str(((testing_latest['Total Cases'] - testing_latest['Deaths']) / testing_latest['Total Cases']).map('{:.2%}'.format)).split()[1] + 
    "</span>" +
    "</div>" +

    "</div>" +

    "<span style='color: black; font-size: 11px;'> Ontario data last updated: " + 
    str(testing_latest['Reported Date'].max().strftime('%Y-%m-%d')) + ", Refreshed on: " + datetime.now(pytz.timezone('America/New_York') ).strftime("%Y-%m-%d at %H:%M") +
    "</span>" +

    "</div>"
    
    ))